In [1]:
import pandas as pd
import numpy as np

In [2]:
#loading the cleaned dataframe for preprocessing

df = pd.read_csv('Peruvian_Bank_Data/clean_df.csv', header = 0)
df.head()

,age,job,marital,education,in_default,avg_yearly_balance,housing_loan,personal_loan,contact_method,day,month,duration,campaign_contacts,prev_days,previous_contacts,prev_outcome,term_deposit
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,5,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,5,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,5,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,5,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,5,198,1,-1,0,unknown,no


I believe the best way to preprocess this is to provide dummy labels to the categorical data and then use a RobustScaler for avg_yearly_balance, and duration because of the outliers. Prev_days will also get a robustscaling, but this will be done last because it is subject to change -- I'm concerned about the negative one being abrasive to the model. But we will see.  MinMax scaler for campaign_contacts and previous_contacts and age. Let's begin and see how it goes. I will do the scaling first and then the dummy variables -- the dummy variables make the dataframe a bit unbearable to scroll through. 

In [3]:
from sklearn.preprocessing import RobustScaler, MinMaxScaler

In [4]:
#applying robustscaler to avg_yearly_balance, duration, prev_days
X = pd.DataFrame(df['avg_yearly_balance'])
RobSca = RobustScaler().fit_transform(X)

In [5]:
df2 = df.copy() #creating a copy just in case something goes wrong
#going off to replace the avg_yearly_balance values
df2['avg_yearly_balance'] = RobSca

In [6]:
df2.head() #looks successful, let's continue. As practice let's create a function for the remaining required preprocessing

,age,job,marital,education,in_default,avg_yearly_balance,housing_loan,personal_loan,contact_method,day,month,duration,campaign_contacts,prev_days,previous_contacts,prev_outcome,term_deposit
0,58,management,married,tertiary,no,1.247241,yes,no,unknown,5,5,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,-0.308315,yes,no,unknown,5,5,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,-0.328182,yes,yes,unknown,5,5,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,0.778514,yes,no,unknown,5,5,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,-0.328918,no,no,unknown,5,5,198,1,-1,0,unknown,no


In [7]:
#creating a function to make the rest of the preprocessing simpler
def preproc(dataframe, column, scalertype):
    X = pd.DataFrame(dataframe[column])
    scaler = scalertype.fit_transform(X)
    dataframe[column] = scaler
    

In [8]:
preproc(df2, 'duration', RobustScaler())

In [9]:
preproc(df2, 'prev_days', RobustScaler())

In [10]:
#minmax for campaign_contacts and previous_contacts and age
preproc(df2, 'campaign_contacts', MinMaxScaler())
preproc(df2, 'previous_contacts', MinMaxScaler())
preproc(df2, 'age', MinMaxScaler())
df2.head()

,age,job,marital,education,in_default,avg_yearly_balance,housing_loan,personal_loan,contact_method,day,month,duration,campaign_contacts,prev_days,previous_contacts,prev_outcome,term_deposit
0,0.519481,management,married,tertiary,no,1.247241,yes,no,unknown,5,5,0.373272,0.0,0.0,0.0,unknown,no
1,0.337662,technician,single,secondary,no,-0.308315,yes,no,unknown,5,5,-0.133641,0.0,0.0,0.0,unknown,no
2,0.194805,entrepreneur,married,secondary,no,-0.328182,yes,yes,unknown,5,5,-0.479263,0.0,0.0,0.0,unknown,no
3,0.376623,blue-collar,married,unknown,no,0.778514,yes,no,unknown,5,5,-0.405530,0.0,0.0,0.0,unknown,no
4,0.194805,unknown,single,unknown,no,-0.328918,no,no,unknown,5,5,0.082949,0.0,0.0,0.0,unknown,no


In [11]:
#decided also here that I should do campaign contacts
preproc(df2, 'campaign_contacts', MinMaxScaler())
df2.head()

,age,job,marital,education,in_default,avg_yearly_balance,housing_loan,personal_loan,contact_method,day,month,duration,campaign_contacts,prev_days,previous_contacts,prev_outcome,term_deposit
0,0.519481,management,married,tertiary,no,1.247241,yes,no,unknown,5,5,0.373272,0.0,0.0,0.0,unknown,no
1,0.337662,technician,single,secondary,no,-0.308315,yes,no,unknown,5,5,-0.133641,0.0,0.0,0.0,unknown,no
2,0.194805,entrepreneur,married,secondary,no,-0.328182,yes,yes,unknown,5,5,-0.479263,0.0,0.0,0.0,unknown,no
3,0.376623,blue-collar,married,unknown,no,0.778514,yes,no,unknown,5,5,-0.405530,0.0,0.0,0.0,unknown,no
4,0.194805,unknown,single,unknown,no,-0.328918,no,no,unknown,5,5,0.082949,0.0,0.0,0.0,unknown,no


In [12]:
#Getting dummies now for the categorical data
#df_ready = this dataframe is ready for modeling
df_ready = pd.get_dummies(df2, columns = ['job','marital','education','in_default','housing_loan','personal_loan','contact_method','prev_outcome'])


In [13]:
df_ready.columns

Index(['age', 'avg_yearly_balance', 'day', 'month', 'duration',
       'campaign_contacts', 'prev_days', 'previous_contacts', 'term_deposit',
       'job_admin.', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_divorced', 'marital_married', 'marital_single',
       'education_primary', 'education_secondary', 'education_tertiary',
       'education_unknown', 'in_default_no', 'in_default_yes',
       'housing_loan_no', 'housing_loan_yes', 'personal_loan_no',
       'personal_loan_yes', 'contact_method_cellular',
       'contact_method_telephone', 'contact_method_unknown',
       'prev_outcome_failure', 'prev_outcome_other', 'prev_outcome_success',
       'prev_outcome_unknown'],
      dtype='object')

### Baseline Models

I will be running three baseline models with default parameters to test the predictive capability. Tune as necessary

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import classification_report

X = df_ready.drop(columns = ['term_deposit'])
y = df_ready['term_deposit']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 44)

In [15]:
#Logistic Regression CV
for i in range(10,21,2):
    clf = LogisticRegressionCV(cv=3, Cs = i, random_state=44, max_iter = 1500).fit(X_train, y_train)
    clf_predictions = clf.predict(X_test)
    clf_train_score = clf.score(X_train, y_train)
    clf_test_score = clf.score(X_test, y_test)
    print("Train Score: {}, Test Score: {} \n".format(clf_train_score, clf_test_score))
    print("Cs: {}. \n{}".format(i, classification_report(y_test, clf_predictions)))

Train Score: 0.8998621165115478, Test Score: 0.9012064343163538 

Cs: 10. 
              precision    recall  f1-score   support

          no       0.91      0.98      0.95     13107
         yes       0.70      0.33      0.45      1813

    accuracy                           0.90     14920
   macro avg       0.81      0.65      0.70     14920
weighted avg       0.89      0.90      0.89     14920

Train Score: 0.8998333907847869, Test Score: 0.9012734584450403 

Cs: 12. 
              precision    recall  f1-score   support

          no       0.91      0.98      0.95     13107
         yes       0.70      0.33      0.45      1813

    accuracy                           0.90     14920
   macro avg       0.81      0.66      0.70     14920
weighted avg       0.89      0.90      0.89     14920

Train Score: 0.8998046650580259, Test Score: 0.9015415549597855 

Cs: 14. 
              precision    recall  f1-score   support

          no       0.91      0.98      0.95     13107
         yes

In [16]:
#the order is no, yes, let's focus on the yes
clfproba = clf.predict_proba(X_test)[:,1]
print(clfproba)
clfprobatrain = clf.predict_proba(X_train)[:,1]
print(clfprobatrain)

[0.0746327  0.6995806  0.02754351 ... 0.06392012 0.66884846 0.01998895]
[0.04230711 0.16225694 0.053066   ... 0.03232473 0.025209   0.06917619]


In [17]:
df_coef = pd.DataFrame(index = X.columns, data = clf.coef_[0, :], columns = ['Coefficient'])
#Here the odds that campaign contacts and previous succeses have an impact on 'yes' are high
df_coef['Abs_Coefficient'] = abs(clf.coef_[0])
df_coef

,Coefficient,Abs_Coefficient
age,0.010840,0.010840
avg_yearly_balance,0.029405,0.029405
day,-0.005246,0.005246
month,-0.017539,0.017539
duration,0.859012,0.859012
campaign_contacts,-4.809073,4.809073
prev_days,-0.000106,0.000106
previous_contacts,0.051427,0.051427
job_admin.,0.193715,0.193715
job_blue-collar,-0.148794,0.148794


In [18]:
clf_predictions = clf.predict(X_test)
clf_train_score = clf.score(X_train, y_train)
clf_test_score = clf.score(X_test, y_test)
print(clf_train_score, clf_test_score)

0.8998621165115478 0.9015415549597855


Random Forests Classification

In [19]:
from sklearn.ensemble import RandomForestClassifier

#To make later parameter tuning easier, let's make sure we check multiple depths. 

for i in range(5,40,5):
    rfc = RandomForestClassifier(max_depth=i, random_state=44).fit(X_train, y_train)
    rfc_predictions = rfc.predict(X_test)
    rfc_train_score = rfc.score(X_train, y_train)
    rfc_test_score = rfc.score(X_test, y_test)
    print("Train Score: {}, Test Score: {} \n".format(rfc_train_score, rfc_test_score))
    print("Depth: {}. \n{}".format(i, classification_report(y_test, rfc_predictions)))

Train Score: 0.8949212915086752, Test Score: 0.8901474530831099 

Depth: 5. 
              precision    recall  f1-score   support

          no       0.89      1.00      0.94     13107
         yes       0.82      0.12      0.21      1813

    accuracy                           0.89     14920
   macro avg       0.86      0.56      0.58     14920
weighted avg       0.88      0.89      0.85     14920

Train Score: 0.9149718487877744, Test Score: 0.8977882037533512 

Depth: 10. 
              precision    recall  f1-score   support

          no       0.90      0.99      0.94     13107
         yes       0.79      0.22      0.34      1813

    accuracy                           0.90     14920
   macro avg       0.84      0.60      0.64     14920
weighted avg       0.89      0.90      0.87     14920

Train Score: 0.963144892565782, Test Score: 0.9109249329758713 

Depth: 15. 
              precision    recall  f1-score   support

          no       0.92      0.98      0.95     13107
     

An important factor here is recall, we want to try to get recall as high as possible so that we are able to correctly identify customers. 0.45 is higher than before, if our precision falters, but our recall is high, this is acceptable as a few misplaced man-hours is less impactful than missed opportunities.

After running more baselines, we will revisit this using the probabilities from predict_proba_ to determine tweak. 

KNN baseline

In [20]:
from sklearn.neighbors import KNeighborsClassifier
#let's do some initial testing with different neighbors
for i in range(2,12,2):
    knn = KNeighborsClassifier(n_neighbors=i).fit(X_train, y_train)
    knn_predictions = knn.predict(X_test)
    knn_train_score = knn.score(X_train, y_train)
    knn_test_score = knn.score(X_test, y_test)
    print("Train Score: {}, Test_Score: {} \n".format(knn_train_score, knn_test_score))
    print("Neighbors: {}. \n{}".format(i, classification_report(y_test, knn_predictions)))

Train Score: 0.9418304033092038, Test_Score: 0.892225201072386 

Neighbors: 2. 
              precision    recall  f1-score   support

          no       0.90      0.98      0.94     13107
         yes       0.64      0.25      0.36      1813

    accuracy                           0.89     14920
   macro avg       0.77      0.62      0.65     14920
weighted avg       0.87      0.89      0.87     14920

Train Score: 0.9235321153625187, Test_Score: 0.8928954423592493 

Neighbors: 4. 
              precision    recall  f1-score   support

          no       0.91      0.98      0.94     13107
         yes       0.64      0.27      0.38      1813

    accuracy                           0.89     14920
   macro avg       0.77      0.62      0.66     14920
weighted avg       0.87      0.89      0.87     14920

Train Score: 0.9159485234976444, Test_Score: 0.8931635388739947 

Neighbors: 6. 
              precision    recall  f1-score   support

          no       0.91      0.98      0.94     1

In [21]:
#simple naive bayes just to see what a very simple model handles.
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()
nb.fit(X_train, y_train)
nb_predictions = nb.predict(X_test)
nb_train_score = nb.score(X_train, y_train)
nb_test_score = nb.score(X_test, y_test)
print("Train Score: {}, Test_Score: {} \n".format(nb_train_score, nb_test_score))
print("\n{}".format(classification_report(y_test, nb_predictions)))


Train Score: 0.8140009192232563, Test_Score: 0.813337801608579 


              precision    recall  f1-score   support

          no       0.94      0.85      0.89     13107
         yes       0.34      0.58      0.43      1813

    accuracy                           0.81     14920
   macro avg       0.64      0.71      0.66     14920
weighted avg       0.86      0.81      0.83     14920



Let's go ahead and tweak the best of each. Again, we're going for high recall.

In [24]:
#Logistic Regression CV, using predict_proba
#testing different thresholds of probability
proba_thresh_list = [0.01, 0.025, 0.05,0.075,0.1,0.15,0.2,0.25,0.3]

for i in proba_thresh_list:
    clf = LogisticRegressionCV(cv=3, Cs = 14, random_state=44, max_iter = 1500).fit(X_train, y_train)
    clf_predictions = clf.predict(X_test)
    clf_train_score = clf.score(X_train, y_train)
    clf_test_score = clf.score(X_test, y_test)
    clf_proba = clf.predict_proba(X_test)
    clf_lower_thresh = (clf_proba[:,1] > i)
    print("Train Score: {}, Test Score: {} \n".format(clf_train_score, clf_test_score))
    print("Probability Threshold: {} \n{}".format(i, classification_report((y_test == 'yes'), clf_lower_thresh)))

Train Score: 0.8998046650580259, Test Score: 0.9015415549597855 

Probability Threshold: 0.01 
              precision    recall  f1-score   support

       False       1.00      0.09      0.17     13107
        True       0.13      1.00      0.23      1813

    accuracy                           0.20     14920
   macro avg       0.57      0.55      0.20     14920
weighted avg       0.89      0.20      0.18     14920

Train Score: 0.8998046650580259, Test Score: 0.9015415549597855 

Probability Threshold: 0.025 
              precision    recall  f1-score   support

       False       0.99      0.31      0.47     13107
        True       0.16      0.99      0.28      1813

    accuracy                           0.39     14920
   macro avg       0.58      0.65      0.37     14920
weighted avg       0.89      0.39      0.44     14920

Train Score: 0.8998046650580259, Test Score: 0.9015415549597855 

Probability Threshold: 0.05 
              precision    recall  f1-score   support

     

As predicted lower threshold, higher recall, but very low precision. Let's continue on and see how the other models fare.

In [25]:
#RandomForests, using predict_proba
#testing different thresholds of probability

for i in proba_thresh_list:
    rfc = RandomForestClassifier(max_depth=15, random_state=44).fit(X_train, y_train)
    rfc_predictions = rfc.predict(X_test)
    rfc_train_score = rfc.score(X_train, y_train)
    rfc_test_score = rfc.score(X_test, y_test)
    rfc_proba = rfc.predict_proba(X_test)
    rfc_lower_thresh = (rfc_proba[:,1] > i).astype('int')
    print("Train Score: {}, Test Score: {} \n".format(rfc_train_score, rfc_test_score))
    print("Probability Threshold: {}. \n{}".format(i, classification_report((y_test == 'yes').astype('int'), rfc_lower_thresh)))

Train Score: 0.963144892565782, Test Score: 0.9109249329758713 

Probability Threshold: 0.01. 
              precision    recall  f1-score   support

           0       1.00      0.24      0.39     13107
           1       0.15      1.00      0.27      1813

    accuracy                           0.33     14920
   macro avg       0.58      0.62      0.33     14920
weighted avg       0.90      0.33      0.37     14920

Train Score: 0.963144892565782, Test Score: 0.9109249329758713 

Probability Threshold: 0.025. 
              precision    recall  f1-score   support

           0       1.00      0.50      0.67     13107
           1       0.22      0.99      0.36      1813

    accuracy                           0.56     14920
   macro avg       0.61      0.75      0.51     14920
weighted avg       0.90      0.56      0.63     14920

Train Score: 0.963144892565782, Test Score: 0.9109249329758713 

Probability Threshold: 0.05. 
              precision    recall  f1-score   support

     

In [26]:
#KNN probability testing

for i in proba_thresh_list:
    knn = KNeighborsClassifier(n_neighbors=8).fit(X_train, y_train)
    knn_predictions = knn.predict(X_test)
    knn_train_score = knn.score(X_train, y_train)
    knn_test_score = knn.score(X_test, y_test)
    knn_proba = knn.predict_proba(X_test)
    knn_lower_thresh = (knn_proba[:,1] > i)
    print("Train Score: {}, Test_Score: {} \n".format(knn_train_score, knn_test_score))
    print("Probability Threshold: {}. \n{}".format(i, classification_report((y_test == 'yes'), knn_lower_thresh)))

Train Score: 0.9113236814891417, Test_Score: 0.8942359249329759 

Probability Threshold: 0.01. 
              precision    recall  f1-score   support

       False       0.98      0.78      0.87     13107
        True       0.35      0.88      0.50      1813

    accuracy                           0.79     14920
   macro avg       0.67      0.83      0.68     14920
weighted avg       0.90      0.79      0.82     14920

Train Score: 0.9113236814891417, Test_Score: 0.8942359249329759 

Probability Threshold: 0.025. 
              precision    recall  f1-score   support

       False       0.98      0.78      0.87     13107
        True       0.35      0.88      0.50      1813

    accuracy                           0.79     14920
   macro avg       0.67      0.83      0.68     14920
weighted avg       0.90      0.79      0.82     14920

Train Score: 0.9113236814891417, Test_Score: 0.8942359249329759 

Probability Threshold: 0.05. 
              precision    recall  f1-score   support

  

It looks as though with this edit, RandomForests is doing the best here. We can see that the with a probability threshold of 0.2 we are getting: 
Probability Threshold: 0.2. 
              precision    recall  f1-score   support

           0       0.97      0.89      0.93     13107
           1       0.51      0.83      0.63      1813
           

This allows shows us that we will be filling our picks just over half with incorrect picks, but we are securing 0.83 of the correct choices.

In sales terms, the average call 4.3 minutes. We currently have a 'yes' rate of 11.68% of customers. But we have 50,000 customers (about) at 4.3 minutes per 1 contact is 215,000 minutes to secure 5840 deposits. Thats 1 term deposit every 36.8 minutes. However if we instead decide to use the model. We will still be securing 4847 deposits. But cutting the outgoing calls to only 9504 customers. At 4.3 minutes avg for one contact, this is 40,867 minutes and a sale every 8.4 minutes. At the cost of 17% of the term deposits, the company can save 174,133 minutes, i.e 2,902 hours of labor-time. Let's say the call center employee costs 20 dollars an hour (total including packages) for the employer. this saves 58, 040 dollars during the campaign time. This number should be compared to the profit that is lost if this model were to be adopted. There may also be other non-material value in each sale, such as further customer loyalty and susceptability to future sales which could be accounted for. 

However, this approach is not binary. Now we can venture forth and create a model that is to tier the employees based on their likelihood to sign up. This can at least become a base from which to spring a campaign. 

Before we do that, I would like to idealize a few options based on this model just to see if there the previous thresholds of 0.15, and 0.1 .etc are plausible towards this goal.

In [36]:
#creating a small function to spit out some stats
#creating tuples of reported info
rf_01 = ("1%", .15, 1.0)
rf_025 = ("2.5%", .22, .99)
rf_05 = ("5%", .3, .98)
rf_075 = ("7.5%", .35, .96)
rf_10 = ("10%", .39, .93)
rf_15 = ("15%", .45, .87)
rf_20 = ('20%', .51, .83)

rf_list = [rf_01, rf_025, rf_05, rf_075, rf_10, rf_15, rf_20]

def trade_off_report(rf_tuple, cust = 50000, conv_perc = 0.1168, avg_dur = 4.3):
    nm_conv = round((cust * conv_perc),2)
    nm_dur_total = round((cust * avg_dur),2)
    nm_conv_interval = round((nm_dur_total / nm_conv),2)
    m_conv = round((nm_conv * rf_tuple[2]),2)
    m_dur_total = round(((m_conv / rf_tuple[1]) * avg_dur),2) #getting all customers that would be called for 'yes', then multiplying by duration
    m_conv_interval = round((m_dur_total / m_conv),2)
    print("Threshold: {}\nNo Model Conversions: {}\nModel Conversions: {}\nNo Model Minutes used: {}\nModel Minutes Used: {}\n\
    No Model Conversion Interval: {}\nModel Conversion Interval: {}\n".format(rf_tuple[0], nm_conv, m_conv, nm_dur_total, m_dur_total, nm_conv_interval, m_conv_interval))


In [31]:
for i in rf_list:
    trade_off_report(rf_tuple = i)

Threshold: 1%
No Model Conversions: 5840.0
Model Conversions: 5840.0
No Model Minutes used: 215000.0
Model Minutes Used: 167413.33
    No Model Conversion Interval: 36.82
Model Conversion Interval: 28.67

Threshold: 2.5
No Model Conversions: 5840.0
Model Conversions: 5781.6
No Model Minutes used: 215000.0
Model Minutes Used: 113004.0
    No Model Conversion Interval: 36.82
Model Conversion Interval: 19.55

Threshold: 5%
No Model Conversions: 5840.0
Model Conversions: 5723.2
No Model Minutes used: 215000.0
Model Minutes Used: 82032.53
    No Model Conversion Interval: 36.82
Model Conversion Interval: 14.33

Threshold: 7.5%
No Model Conversions: 5840.0
Model Conversions: 5606.4
No Model Minutes used: 215000.0
Model Minutes Used: 68878.63
    No Model Conversion Interval: 36.82
Model Conversion Interval: 12.29

Threshold: 10%
No Model Conversions: 5840.0
Model Conversions: 5431.2
No Model Minutes used: 215000.0
Model Minutes Used: 59882.46
    No Model Conversion Interval: 36.82
Model Con

this shows that even with a model threshold at 0.01%, the company can still save time: 47587 speculated minutes. And this threshold would lose 0 sales. Even the safest threshold allows us to maintain sales. However, I have to wonder. is there a threshold between 1% and 2.5% that allows us to maximize? No point in wondering for too long, let's put it the test. 

In [34]:
#testing a tight range of threshold amounts

for i in np.arange(0.01, 0.025, 0.001): 
    rfc = RandomForestClassifier(max_depth=15, random_state=44).fit(X_train, y_train)
    rfc_predictions = rfc.predict(X_test)
    rfc_train_score = rfc.score(X_train, y_train)
    rfc_test_score = rfc.score(X_test, y_test)
    rfc_proba = rfc.predict_proba(X_test)
    rfc_lower_thresh = (rfc_proba[:,1] > i).astype('int')
    print("Train Score: {}, Test Score: {} \n".format(rfc_train_score, rfc_test_score))
    print("Probability Threshold: {}. \n{}".format(i, classification_report((y_test == 'yes').astype('int'), rfc_lower_thresh)))
    
#Note: this is slow, but necessary if we want to be absolutely sure of our maximization, each call saved adds up quickly.

Train Score: 0.963144892565782, Test Score: 0.9109249329758713 

Probability Threshold: 0.01. 
              precision    recall  f1-score   support

           0       1.00      0.24      0.39     13107
           1       0.15      1.00      0.27      1813

    accuracy                           0.33     14920
   macro avg       0.58      0.62      0.33     14920
weighted avg       0.90      0.33      0.37     14920

Train Score: 0.963144892565782, Test Score: 0.9109249329758713 

Probability Threshold: 0.011. 
              precision    recall  f1-score   support

           0       1.00      0.26      0.41     13107
           1       0.16      1.00      0.27      1813

    accuracy                           0.35     14920
   macro avg       0.58      0.63      0.34     14920
weighted avg       0.90      0.35      0.40     14920

Train Score: 0.963144892565782, Test Score: 0.9109249329758713 

Probability Threshold: 0.011999999999999999. 
              precision    recall  f1-score 

Looks like the drop off is right at 1.8% let's see how this compares to 1% and 2.5%

In [37]:
rf_018 = ("1.8%", 0.19, 1.0)
rf_tight_list = [rf_01, rf_018, rf_025]

for i in rf_tight_list:
    trade_off_report(rf_tuple = i)

Threshold: 1%
No Model Conversions: 5840.0
Model Conversions: 5840.0
No Model Minutes used: 215000.0
Model Minutes Used: 167413.33
    No Model Conversion Interval: 36.82
Model Conversion Interval: 28.67

Threshold: 1.8%
No Model Conversions: 5840.0
Model Conversions: 5840.0
No Model Minutes used: 215000.0
Model Minutes Used: 132168.42
    No Model Conversion Interval: 36.82
Model Conversion Interval: 22.63

Threshold: 2.5%
No Model Conversions: 5840.0
Model Conversions: 5781.6
No Model Minutes used: 215000.0
Model Minutes Used: 113004.0
    No Model Conversion Interval: 36.82
Model Conversion Interval: 19.55



So if it was decided that no conversion loss was wanted, we could propose this model at a threshold of 1.8% and still reduce our minutes down by 52,832 i.e 880 hours. I believe that this type of improvement would be very benefical in maximizing worker-hours. We could cut more if we wanted to, it depends on how the bank is willing to balance hours spent with the response rate.


As promised let's dig into the tiers. Based on the thresholds. We can use the random forest with the whole dataframe for this

In [91]:
rfc_whole_proba = rfc.predict_proba(df_ready.drop(columns ='term_deposit'))
df_copy = df.copy()
df_copy['probability'] = rfc_whole_proba[:,1]
df_copy['probability'] = df_copy['probability'].round(3)
df_copy.head()

,age,job,marital,education,in_default,avg_yearly_balance,housing_loan,personal_loan,contact_method,day,month,duration,campaign_contacts,prev_days,previous_contacts,prev_outcome,term_deposit,probability
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,5,261,1,-1,0,unknown,no,0.006
1,44,technician,single,secondary,no,29,yes,no,unknown,5,5,151,1,-1,0,unknown,no,0.003
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,5,76,1,-1,0,unknown,no,0.008
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,5,92,1,-1,0,unknown,no,0.005
4,33,unknown,single,unknown,no,1,no,no,unknown,5,5,198,1,-1,0,unknown,no,0.004


In [103]:
#Tiering thet dataset
df_copy['tier'] = pd.cut(df_copy['probability'],[0, 0.018, 0.4, 0.6, 1.0],labels=['tier_4', 'tier_3', 'tier_2', 'tier_1'])

In [106]:
df_copy.tier.loc[df_copy['term_deposit'] == 'yes'].value_counts().sort_index()

tier_4       9
tier_3    1548
tier_2    1828
tier_1    2425
Name: tier, dtype: int64

So it looks like we have a few lost travellers here. But we can attribute this to a reworking of the machine learning algorithm using all of the data rather than prior with just the training, then test set. missing 9 conversions is negligble in the grand scheme as we would still be saving a lot of time.

In [107]:
df_copy.tier.loc[df_copy['term_deposit'] == 'no'].value_counts().sort_index()

tier_4    19648
tier_3    23621
tier_2      574
tier_1       75
Name: tier, dtype: int64

Here we see that if we wanted to completely avoid calling tier_4 customers. We would be be reducing the customers contacted by 19857, only 9 of which, in this case, subscribed to a term deposit. So as you can see below, about half a percent of those customers were likely to sign up. Let's look at the chance of calling each tier based on their current success rate

In [115]:
print("Tier 4: " + str(round((9/19657),4)))
print("Tier 3: " + str(round((1548/25169),4)))
print("Tier 2: " + str(round((1828/2402),4)))
print("Tier 1: " + str(round((2425/2500),4)))

Tier 4: 0.0005
Tier 3: 0.0615
Tier 2: 0.761
Tier 1: 0.97


By tiering the customers in this way, we can strike hardest where the customer is most likely to respond well. To finalize this segment, let's go ahead and print what an ideal dataframe would look like before a campaign.

In [118]:
df_copy['tier'].head(10) #and that's it.

0    tier_4
1    tier_4
2    tier_4
3    tier_4
4    tier_4
5    tier_4
6    tier_4
7    tier_4
8    tier_4
9    tier_4
Name: tier, dtype: category
Categories (4, object): [tier_4 < tier_3 < tier_2 < tier_1]